In [3]:
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller



In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar el archivo depurado.csv
data = pd.read_csv('depurado.csv')

# Convertir la columna de fecha a tipo datetime
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Ordenar los datos por fecha
data = data.sort_values('Fecha')

# Seleccionar las columnas de interés
columns_interes = [
    'Diesel bajo azufre',
    'Diesel ultra bajo azufre',
    'Gas licuado de petróleo',
    'Gasolina regular',
    'Gasolina superior',
    'Diesel alto azufre'
]

# Resamplear los datos a un período mensual y sumar los valores para obtener el consumo mensual
monthly_data = data.set_index('Fecha').resample('M').sum()

# Separar los datos en entrenamiento y prueba
train_data, test_data = train_test_split(monthly_data, test_size=0.2, shuffle=False)

# Mostrar las primeras filas del conjunto de entrenamiento
print(train_data.head())


            Unnamed: 0  Aceites lubricantes  Asfalto      Bunker  Ceras  \
Fecha                                                                     
2001-01-31           0                  0.0  27748.99  214581.84    0.0   
2001-02-28           1                  0.0   7503.57  294609.00    0.0   
2001-03-31           2                  0.0  26304.32  315263.80    0.0   
2001-04-30           3                  0.0   7885.89  205653.00    0.0   
2001-05-31           4                  0.0   8443.16  278371.30    0.0   

            Combustible turbo jet  Diesel bajo azufre  \
Fecha                                                   
2001-01-31                    0.0                 0.0   
2001-02-28                    0.0                 0.0   
2001-03-31                    0.0                 0.0   
2001-04-30                    0.0                 0.0   
2001-05-31                    0.0                 0.0   

            Diesel ultra bajo azufre  Gas licuado de petróleo  \
Fecha    

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Escalar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)

# Crear secuencias para el LSTM
def create_sequences(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), :])
        y.append(data[i + time_steps, :])
    return np.array(X), np.array(y)

time_steps = 10
X_train, y_train = create_sequences(train_scaled, time_steps)
X_test, y_test = create_sequences(test_scaled, time_steps)

# Primer modelo LSTM
model_1 = Sequential()
model_1.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_1.add(Dropout(0.2))
model_1.add(LSTM(50, return_sequences=False))
model_1.add(Dropout(0.2))
model_1.add(Dense(y_train.shape[1]))
model_1.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el primer modelo
history_1 = model_1.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Segundo modelo LSTM (diferente configuración)
model_2 = Sequential()
model_2.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_2.add(Dropout(0.3))
model_2.add(LSTM(100, return_sequences=False))
model_2.add(Dropout(0.3))
model_2.add(Dense(y_train.shape[1]))
model_2.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el segundo modelo
history_2 = model_2.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
